## Grupo Bimbo Inventory Demand
https://www.kaggle.com/c/grupo-bimbo-inventory-demand

E-commerce website: http://blumonpay.net/bimbo/

In [ ]:
dir = "C:/github/kaggle-fiddling/bimbo/"

In [1]:
import numpy as np 
import pandas as pd 
import bcolz

import ipython_memory_usage as imu
imu.start_watching_memory()

bcolz.blosc_set_nthreads(8)

In [1] used 0.0156 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 74.07 MiB


In [2]:
# read from CSV ; comment-in next lines when done
train = pd.read_csv(dir + 'train.csv', dtype={
    'Semana': np.int8,
    'Agencia_ID': np.int32,
    'Canal_ID': np.int16,
    'Ruta_SAK': np.int16,
    'Cliente_ID': np.int64,
    'Producto_ID': np.int32,
    'Venta_uni_hoy': np.int16,
    'Venta_hoy': np.float16,
    'Dev_uni_proxima': np.int32,
    'Dev_proxima': np.float16,
    'Demanda_uni_equil': np.int16
})

In [2] used 2337.8398 MiB RAM in 48.34s, peaked 4397.83 MiB above current, total RAM usage 2411.91 MiB


In [3]:
# generate bcolz file; comment-in next lines when done
zTrain = bcolz.ctable.fromdataframe(train)
zTrain.copy(rootdir=dir+"train.bcolz", mode='w')

ctable((74180464,), [('Semana', 'i1'), ('Agencia_ID', '<i4'), ('Canal_ID', '<i2'), ('Ruta_SAK', '<i2'), ('Cliente_ID', '<i8'), ('Producto_ID', '<i4'), ('Venta_uni_hoy', '<i2'), ('Venta_hoy', '<f2'), ('Dev_uni_proxima', '<i4'), ('Dev_proxima', '<f2'), ('Demanda_uni_equil', '<i2')])
  nbytes: 2.28 GB; cbytes: 586.13 MB; ratio: 3.98
  cparams := cparams(clevel=5, shuffle=1, cname='blosclz')
  rootdir := 'C:/github/kaggle-fiddling/bimbo/train.bcolz'
[(3, 1110, 7, 3301, 15766, 1212, 3, 25.140625, 0, 0.0, 3)
 (3, 1110, 7, 3301, 15766, 1216, 4, 33.53125, 0, 0.0, 4)
 (3, 1110, 7, 3301, 15766, 1238, 4, 39.3125, 0, 0.0, 4) ...,
 (9, 25759, 1, 5517, 4528866, 45112, 4, 51.0, 0, 0.0, 4)
 (9, 25759, 1, 5517, 4547943, 40217, 3, 26.9375, 3, 26.9375, 0)
 (9, 25759, 1, 5517, 4708097, 43159, 1, 10.3984375, 0, 0.0, 1)]

In [3] used 596.2227 MiB RAM in 11.36s, peaked 0.00 MiB above current, total RAM usage 3008.14 MiB


In [6]:
train[['Semana','Agencia_ID']].groupby('Semana').count()

,Agencia_ID
Semana,
3,11165207
4,11009593
5,10615397
6,10191837
7,10382849
8,10406868
9,10408713


In [6] used 0.0547 MiB RAM in 2.60s, peaked 2349.43 MiB above current, total RAM usage 3008.50 MiB


In [7]:
%%python2
print 'ok'

Couldn't find program: 'python2'
In [7] used 0.0391 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 3008.54 MiB


In [ ]:
import dataextract as tde
#module os - to delete files when we need to
import csv,os,datetime
import locale
import os.path
import time
locale.setlocale( locale.LC_ALL, 'English_United States.1252' )
 
#Step 1: Create the extract file
 
# if running the script multiple times
# path - name of the extract
path = dir+'train.tde'
 
#if the file exists and is readable then it creates new file
if os.path.isfile(PATH) and os.access(PATH, os.R_OK):
   # File exists and is readable
    os.remove(path)
    tdefile = tde.Extract(path)
 
else:
   # Either file is missing or is not readable - create new file
    tdefile = tde.Extract(path)
 
 
# open csv file
csvReader = csv.reader(open(dir+'train.csv','rb'), delimiter=';',quotechar='"')
 
#Step 2: Create the tabledef
tableDef = tde.TableDefinition()
tableDef.addColumn('Semana', tde.Type.INTEGER) 
tableDef.addColumn('Agencia_ID', tde.Type.INTEGER) 
tableDef.addColumn('Canal_ID', tde.Type.INTEGER) 
tableDef.addColumn('Ruta_SAK', tde.Type.INTEGER) 
tableDef.addColumn('Cliente_ID', tde.Type.INTEGER)
tableDef.addColumn('Producto_ID', tde.Type.INTEGER) 
tableDef.addColumn('Venta_uni_hoy', tde.Type.INTEGER) 
tableDef.addColumn('Venta_hoy', tde.Type.DOUBLE) 
tableDef.addColumn('Dev_uni_proxima', tde.Type.INTEGER) 
tableDef.addColumn('Dev_proxima', tde.Type.DOUBLE) 
tableDef.addColumn('Demanda_uni_equil Name', tde.Type.INTEGER) 

        
#func returns the proper float number
def floatNum( num ):
   return float(num.replace(",", "."))
 
 
#Step 3: Create the table in the image of the tableDef
table = tdefile.addTable('Extract',tableDef)
 
#Step 4: Loop through the csv, grab all the data, put it into rows
#and insert the rows in the table
newrow = tde.Row(tableDef)
#going to the next row (skip the headers, take the data only
csvReader.next()
for line in csvReader:
    #           col numer(extract), col number(CSV)
    newrow.setCharString(0, str(line[0]))
    newrow.setCharString(1, str(line[1]))
    newrow.setDouble(2, floatNum((line[2])))
    newrow.setDouble(3, floatNum((line[3])))
    newrow.setCharString(4, str(line[9]))
    # save date in a variable (date)
    date = datetime.datetime.strptime(line[18], '%d.%m.%Y')
    newrow.setDate(5, date.year, date.month, date.day)
    newrow.setDouble(6, floatNum((line[22])))
    newrow.setDouble(7, floatNum((line[20])))
    newrow.setInteger(8, int(line[17]))
    newrow.setCharString(9, str(line[6]))
    table.insert(newrow)
 
#Step 5: Close the tde
tdefile.close()